In [1]:
import mysql.connector
# I know this is bad practice, but I did not want to explicitly import every funcion I am using
# Maybe refactoring would be in order for a real world project, but I think it will suffice in this case
from data.execution import *

##### The below will ask for two inputs Database User and Password for local host respectively


In [2]:
cnx, cursor, prep_cursor = init_database()

In [3]:
insert_data(cnx, cursor)

Now using little_lemon_capstone
Check if Data is Already Added


##### Adding Sales Reports


**Creating and Calling the Virtual Tables**


In [4]:
order_views, over90orders, popular_items = run_views(cursor)

In [5]:
print("Order Views Response")
for data in order_views:
    print(
        f"orderId: {data[0]} | Order Quantity: {data[1]} | Order Total: ${data[2]}")

Order Views Response
orderId: 1 | Order Quantity: 12 | Order Total: $86.41
orderId: 2 | Order Quantity: 11 | Order Total: $94.42
orderId: 3 | Order Quantity: 8 | Order Total: $68.45
orderId: 4 | Order Quantity: 7 | Order Total: $96.93
orderId: 5 | Order Quantity: 7 | Order Total: $34.95
orderId: 6 | Order Quantity: 5 | Order Total: $37.46
orderId: 7 | Order Quantity: 7 | Order Total: $39.46
orderId: 8 | Order Quantity: 14 | Order Total: $120.94


In [6]:
print("Orders Over 90 Dollars")
for data in over90orders:
    print(
        f"Customer: {data[1]} | Order Total: ${data[4]} | Order Type: {data[3]}")

Orders Over 90 Dollars
Customer: Jane Smith | Order Total: $94.42 | Order Type: Booking
Customer: Lisa White | Order Total: $96.93 | Order Type: Delivery
Customer: Lisa White | Order Total: $120.94 | Order Type: Walk-in


In [7]:
print("Popular Menu Items")
for data in popular_items:
    print(f'Item: {data[0]}')

Popular Menu Items
Item: Bruschetta
Item: Caprese Salad
Item: Baklava
Item: Galaktoboureko


**Creating and Calling Procedures**


In [8]:
run_proceedures(cursor)

In [9]:
query = call_procedure(cursor, 'GetMaxQuantity')
print(f"The Maximum Order Size is {query[0][0][0]}")

The Maximum Order Size is 14


In [10]:
query = call_procedure(cursor, 'CancelOrder', (9, 'message'))
query

[[('Order 9 is cancelled',)]]

In [11]:
response = run_prepared_statement(prep_cursor, (4, ))
for order in response:
    print(f"Customer #: {order[0]} | Quantity: {order[1]} | Total: {order[3]}")

Customer #: 4 | Quantity: 4 | Total: 57.96
Customer #: 4 | Quantity: 8 | Total: 64.97


##### Table Booking System


In [12]:
create_booking_procedures(cursor)

In [13]:
call_procedure(cursor, "CheckBooking")

[[(1, datetime.datetime(2023, 11, 20, 0, 0), 5, 1),
  (2, datetime.datetime(2023, 11, 21, 0, 0), 3, 2),
  (3, datetime.datetime(2023, 11, 22, 0, 0), 1, 3),
  (4, datetime.datetime(2023, 11, 23, 0, 0), 6, 4)]]

In [16]:
add_booking = call_procedure(
    cursor, "AddBooking", (5, 2, 7, '2023-11-25', 12, 5))
update_booking = call_procedure(cursor, "UpdateBooking", (5, '2023-11-27'))
cancel_booking = call_procedure(cursor, "CancelBooking", (5,))

In [17]:
print("Adding Booking... \n", add_booking)
print("Updating Booking... \n", update_booking)
print("Canceling Booking... \n", cancel_booking)

Adding Booking... 
 [[(5, 2, 7, datetime.datetime(2023, 11, 25, 0, 0), 12, 5)]]
Updating Booking... 
 [[(5, 2, 7, datetime.datetime(2023, 11, 27, 0, 0), 12, 5)]]
Canceling Booking... 
 [[('Booking Id: 5 deleted successfully',)]]


##### Data Analysis Testing


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("./data/lemon_dat.xlsx")

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.rename(columns={" Cost": "Cost"}, inplace=True)
df.head(3)

##### Feature Engineering


In [ ]:
df[["First Name", "Last Name"]] = df["Customer Name"].str.split(
    " ", expand=True, n=1)

In [ ]:
df["Profit"] = df["Sales"] - df["Cost"]

In [ ]:
df[["Quantity", "Profit"]]

##### Building the Visualizations

**Using Pandas for Data Cleaning**

**Using Plotly for Data Visualizations**


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
config = {
    'modeBarButtonsToRemove': ['zoom', 'zoomIn', 'zoomOut', 'select', 'lasso', "autoScale2d"],
}

**Task 1**


In [ ]:
# Data Cleaning
cust_sales = df.copy()
cust_sales = cust_sales.loc[cust_sales["Sales"] >= 70]
cust_sales = cust_sales.groupby("Customer Name", as_index=False).sum(
    numeric_only=True)[["Customer Name", "Sales"]].sort_values("Sales", ascending=False)
cust_sales.head(2)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=cust_sales["Customer Name"],
    y=cust_sales["Sales"],
    hovertemplate="""
    <b>Customer:</b> %{x}<br>
    <b>Total Sales:</b> %{y:$,.2f}
    <extra></extra>
    """,
    texttemplate="%{y:$,.2f}"
))


fig.update_xaxes(range=[-1, 10])
fig.update_yaxes(fixedrange=True)

fig.update_layout(
    dragmode="pan",
    title='Customer Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($ USD)'
)

fig.show(config=config)

**Task 2**


In [ ]:
# Filtering the data
profit_df = df.copy()
profit_df = profit_df.groupby("Order Date", as_index=False).sum(
    numeric_only=True).sort_values("Order Date")
profit_df = profit_df.loc[profit_df["Order Date"] < "2023-01-01"]

In [ ]:
profit_df

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=profit_df["Order Date"],
    y=profit_df["Profit"],
    mode="lines",
    hovertemplate="""
    <b>Profit:</b> %{y:$,.2f}<br>
    <b>Date:</b> %{x}
    <extra></extra>
    """
))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector_buttons=list([
        dict(count=1, label="1m", step="month", stepmode="backward"),
        dict(count=3, label="3m", step="month", stepmode="backward"),
        dict(count=6, label="6m", step="month", stepmode="backward"),
        dict(count=1, label="1y", step="year", stepmode="backward"),
        dict(count=2, label="2yr", step="year", stepmode="backward"),
        dict(count=3, label="3yr", step="year", stepmode="backward"),
    ]),
)

fig.update_layout(
    dragmode="pan",
    title='Profit Chart',
    xaxis_title='Order Date',
    yaxis_title='Total Profit ($ USD)',
    yaxis_tickformat='$,.2f'
)

fig.show(config=config)

**Task 3**


In [ ]:
bubble_df = df.copy()
bubble_df = bubble_df.groupby(
    "Customer Name", as_index=False).sum(numeric_only=True)

In [ ]:
bubble_df

In [ ]:
fig = px.scatter(
    bubble_df,
    x="Profit",
    y="Sales",
    color="Customer Name",
    size="Quantity",
    hover_data={
        "Customer Name": False,
        "Profit": ":$,.2f",
        "Sales": ":$,.2f",
    },
    hover_name="Customer Name"
)

fig.update_layout(
    showlegend=False,
    title='Sales Buble Chart (w/ size=Quantity)',
    yaxis_title='Total Sales ($ USD)',
    yaxis_tickformat="$,.2f",
    xaxis_title='Total Profit ($ USD)',
    xaxis_tickformat="$,.2f"
)

fig.show(config={
    'modeBarButtonsToRemove': ['zoomIn', 'zoomOut', 'select', 'lasso', "autoScale2d"],
})

**Task 4**


In [ ]:
cuisine_df = df.copy()
cuisine_df["Year"] = cuisine_df["Order Date"].dt.year
cuisine_df = cuisine_df.loc[cuisine_df["Year"].between(
    2020, 2022, inclusive="both")]
cuisine_df = cuisine_df.groupby(["Cuisine Name", "Year"], as_index=False).sum(
    numeric_only=True).sort_values(["Year", "Sales"], ascending=False)

In [ ]:
fig = px.histogram(
    cuisine_df,
    x="Year",
    y="Profit",
    color="Cuisine Name",
    barmode='group',
    hover_data={
        "Cuisine Name": False,
        "Year": True,
        "Profit": ":$,.2f"
    },
    hover_name="Cuisine Name"
)

fig.update_traces(texttemplate="%{y:$,.2f}")

fig.update_layout(
    title='Yearly Profit by Cuisine',
    yaxis_title='Total Profit ($ USD)',
    yaxis_tickformat="$,.2f",
    xaxis_title='Year'
)

fig.show(config={
    'modeBarButtonsToRemove': ['zoomIn', 'zoomOut', 'select', 'lasso', "autoScale2d"],
})

**Task 5**


In [ ]:
df.columns

In [ ]:
test = df.groupby(["Customer Name", "Country", "City", "Order Date",
                  "Order ID"], as_index=False).sum(numeric_only=True)
test[test["Customer Name"] == "Doug Logg"]